In [12]:
import numpy as np
import random
import gensim
from nltk.corpus import words
from functools import reduce
import heapq
import os
import collections
from sklearn.cluster import KMeans
from sklearn import metrics

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.300d.w2vformat.txt', binary=False)

In [116]:
words_upper = ["Hollywood", "Well", "Foot", "New York", "Spring", "Court", "Tube", "Point", "Tablet", "Slip", "Date", "Drill", "Lemon", "Bell", "Screen", "Fair", "Torch", "State", "Match", "Iron", "Block", "France", "Australia", "Limousine", "Stream", "Glove", "Nurse", "Leprechaun", "Play", "Tooth", "Arm", "Bermuda", "Diamond", "Whale", "Comic", "Mammoth", "Green", "Pass", "Missile", "Paste", "Drop", "Pheonix", "Marble", "Staff", "Figure", "Park", "Centaur", "Shadow", "Fish", "Cotton", "Egypt", "Theater", "Scale", "Fall", "Track", "Force", "Dinosaur", "Bill", "Mine", "Turkey", "March", "Contract", "Bridge", "Robin", "Line", "Plate", "Band", "Fire", "Bank", "Boom", "Cat", "Shot", "Suit", "Chocolate", "Roulette", "Mercury", "Moon", "Net", "Lawyer", "Satellite", "Angel", "Spider", "Germany", "Fork", "Pitch", "King", "Crane", "Trip", "Dog", "Conductor", "Part", "Bugle", "Witch", "Ketchup", "Press", "Spine", "Worm", "Alps", "Bond", "Pan", "Beijing", "Racket", "Cross", "Seal", "Aztec", "Maple", "Parachute", "Hotel", "Berry", "Soldier", "Ray", "Post", "Greece", "Square", "Mass", "Bat", "Wave", "Car", "Smuggler", "England", "Crash", "Tail", "Card", "Horn", "Capital", "Fence", "Deck", "Buffalo", "Microscope", "Jet", "Duck", "Ring", "Train", "Field", "Gold", "Tick", "Check", "Queen", "Strike", "Kangaroo", "Spike", "Scientist", "Engine", "Shakespeare", "Wind", "Kid", "Embassy", "Robot", "Note", "Ground", "Draft", "Ham", "War", "Mouse", "Center", "China", "Bolt", "Spot", "Piano", "Pupil", "Plot", "Lion", "Police", "Head", "Litter", "Concert", "Mug", "Vacuum", "Atlantis", "Straw", "Switch", "Skyscraper", "Laser", "Scuba Diver", "Africa", "Plastic", "Dwarf", "Lap", "Life", "Honey", "Horseshoe", "Unicorn", "Spy", "Pants", "Wall", "Paper", "Sound", "Ice", "Tag", "Web", "Fan", "Orange", "Temple", "Canada", "Scorpion", "Undertaker", "Mail", "Europe", "Soul", "Apple", "Pole", "Tap", "Mouth", "Ambulance", "Dress", "IceCream", "Rabbit", "Buck", "Agent", "Sock", "Nut", "Boot", "Ghost", "Oil", "Superhero", "Code", "Kiwi", "Hospital", "Saturn", "Film", "Button", "Snowman", "Helicopter", "Log", "Princess", "Time", "Cook", "Revolution", "Shoe", "Mole", "Spell", "Grass", "Washer", "Game", "Beat", "Hole", "Horse", "Pirate", "Link", "Dance", "Fly", "Pit", "Server", "School", "Lock", "Brush", "Pool", "Star", "Jam", "Organ", "Berlin", "Face", "Luck", "Amazon", "Cast", "Gas", "Club", "Sink", "Water", "Chair", "Shark", "Jupiter", "Copper", "Jack", "Platypus", "Stick", "Olive", "Grace", "Bear", "Glass", "Row", "Pistol", "London", "Rock", "Van", "Vet", "Beach", "Charge", "Port", "Disease", "Palm", "Moscow", "Pin", "Washington", "Pyramid", "Opera", "Casino", "Pilot", "String", "Night", "Chest", "Yard", "Teacher", "Pumpkin", "Thief", "Bark", "Bug", "Mint", "Cycle", "Telescope", "Calf", "Air", "Box", "Mount", "Thumb", "Antarctica", "Trunk", "Snow", "Penguin", "Root", "Bar", "File", "Hawk", "Battery", "Compound", "Slug", "Octopus", "Whip", "America", "Ivory", "Pound", "Sub", "Cliff", "Lab", "Eagle", "Genius", "Ship", "Dice", "Hood", "Heart", "Novel", "Pipe", "Himalayas", "Crown", "Round", "India", "Needle", "Shop", "Watch", "Lead", "Tie", "Table", "Cell", "Cover", "Czech", "Back", "Bomb", "Ruler", "Forest", "Bottle", "Space", "Hook", "Doctor", "Ball", "Bow", "Degree", "Rome", "Plane", "Giant", "Nail", "Dragon", "Stadium", "Flute", "Carrot", "Wake", "Fighter", "Model", "Tokyo", "Eye", "Mexico", "Hand", "Swing", "Key", "Alien", "Tower", "Poison", "Cricket", "Cold", "Knife", "Church", "Board", "Cloak", "Ninja", "Olympus", "Belt", "Light", "Death", "Stock", "Millionaire", "Day", "Knight", "Pie", "Bed", "Circle", "Rose", "Change", "Cap", "Triangle", "Chick"]
words = [x.lower() for x in words_upper]
board = random.sample(words, 25)

player1 = board[:9]
player2 = board[9:17]
neutral = board[17:24]
assassin = board[24]

In [117]:
positives = model[player1]

In [118]:
# https://stackoverflow.com/questions/40828929/sklearn-mean-distance-from-centroid-of-each-cluster

kmeans1 = KMeans(n_clusters=5)
clusters1 = kmeans1.fit_predict(positives)
centroids1 = kmeans1.cluster_centers_

mean_distances1 = {}
mean_count1 = {}

kmeans2 = KMeans(n_clusters=4)
clusters2 = kmeans2.fit_predict(positives)
centroids2 = kmeans2.cluster_centers_

mean_distances2 = {}
mean_count2 = {}

kmeans3 = KMeans(n_clusters=4)
clusters3 = kmeans3.fit_predict(positives)
centroids3 = kmeans3.cluster_centers_

mean_distances3 = {}
mean_count3 = {}

for i in range(5):
    mean_distances1[i] = 0
    mean_count1[i] = 0

for i in range(4):
    mean_distances2[i] = 0
    mean_count2[i] = 0

for i in range(4):
    mean_distances3[i] = 0
    mean_count3[i] = 0

# go through clusters and calculate distance to centroid for each
for i in range(9):
    cluster1 = clusters1[i]
    centroid1 = centroids1[cluster1]
    distance1 = np.linalg.norm(centroid1-positives[i])
    mean_distances1[cluster1] += distance1
    mean_count1[cluster1] += 1
    
    cluster2 = clusters2[i]
    centroid2 = centroids2[cluster2]
    distance2 = np.linalg.norm(centroid2-positives[i])
    mean_distances2[cluster2] += distance2
    mean_count2[cluster2] += 1
    
    cluster3 = clusters3[i]
    centroid3 = centroids3[cluster3]
    distance3 = np.linalg.norm(centroid3-positives[i])
    mean_distances3[cluster3] += distance3
    mean_count3[cluster3] += 1

mean_cluster1 = []
mean_cluster2 = []
mean_cluster3 = []

# calculate mean
for i in range(5):
    mean_cluster1.append(mean_distances1[i]/mean_count1[i])
    
for i in range(4):
    mean_cluster2.append(mean_distances2[i]/mean_count2[i])
    
for i in range(4):
    mean_cluster3.append(mean_distances3[i]/mean_count3[i])
    
print mean_cluster1
print mean_distances1
print mean_count1

print mean_cluster2
print mean_distances2
print mean_count2

print mean_cluster3
print mean_distances3
print mean_count3

[1.5469184688754467e-07, 4.546929359436035, 4.608146667480469, 4.476967493693034, 1.2353920908481086e-07]
{0: 1.5469184688754467e-07, 1: 9.09385871887207, 2: 9.216293334960938, 3: 13.430902481079102, 4: 1.2353920908481086e-07}
{0: 1, 1: 2, 2: 2, 3: 3, 4: 1}
[5.309289773305257, 4.498868227005005, 4.476967493693034, 1.2353920908481086e-07]
{0: 15.927869319915771, 1: 8.99773645401001, 2: 13.430902481079102, 3: 1.2353920908481086e-07}
{0: 3, 1: 2, 2: 3, 3: 1}
[1.7442897615183028e-07, 5.360436248779297, 4.546929359436035, 1.2353920908481086e-07]
{0: 1.7442897615183028e-07, 1: 26.802181243896484, 2: 9.09385871887207, 3: 1.2353920908481086e-07}
{0: 1, 1: 5, 2: 2, 3: 1}


In [119]:
# other choice: find largest count

combined = [0, clusters1, clusters2, clusters3]

largest_count = 0
count_value = 0
count_index = 0
trial = 0

for index in range(5):
    if mean_count1[index] > largest_count:
        largest_count = mean_count1[index]
        count_value = mean_cluster1[index]
        count_index = index
        trial = 1
    if mean_count1[index] == largest_count:
        if mean_cluster1[index] < count_value:
            count_index = index
            count_value = mean_cluster1[index]
            trial = 1
        
for index in range(4):
    if mean_count2[index] > largest_count:
        largest_count = mean_count2[index]
        count_value = mean_cluster2[index]
        count_index = index
        trial = 2
    if mean_count2[index] == largest_count:
        if mean_cluster2[index] < count_value:
            count_index = index
            count_value = mean_cluster2[index]
            trial = 2
            
    if mean_count3[index] > largest_count:
        largest_count = mean_count3[index]
        count_value = mean_cluster3[index]
        count_index = index
        trial = 3
    if mean_count3[index] == largest_count:
        if mean_cluster3[index] < count_value:
            count_index = index
            count_value = mean_cluster3[index]
            trial = 3

print largest_count
print count_value
print count_index
print trial

words = []
print combined[trial]
indices = [i for i, e in enumerate(combined[trial]) if e == count_index]
for i in indices:
    words.append(player1[i])
    
print words

5
5.360436248779297
1
3
[1 1 3 2 1 1 2 1 0]
['hospital', 'fall', 'theater', 'box', 'thumb']


In [120]:
full_hint = model.most_similar(
    positive=words,
    negative=player2 + [assassin],
    restrict_vocab=50000
)

In [121]:
index = 0
all_checked = False

for i, word in enumerate(board):
#         print word
#         print full_hint[index][0]
    if full_hint[index][0] in word or word in full_hint[index][0]:
        index += 1
        i = 0
    
hint = full_hint[index][0]
print hint

anemic


In [122]:
print assassin

scientist


In [123]:
similarities = []
for word in board:
    prob = model.similarity(hint, word)
    similarities.append((word, prob))
    
sort_by_similarity = sorted(similarities, key=lambda tup: tup[1], reverse=True)
print sort_by_similarity[:largest_count]

[('fall', 0.13758352), ('ketchup', 0.089986965), ('thumb', 0.057153333), ('box', 0.042899765), ('germany', 0.029806081)]


In [124]:
print words

['hospital', 'fall', 'theater', 'box', 'thumb']
